In [ ]:
!pip install dask

In [ ]:
import dask.dataframe as dd
import pandas as pd

# Load the datasets
phishing_urls = dd.read_csv('/content/drive/MyDrive/phishing.csv')
legitimate_urls = dd.read_csv('/content/drive/MyDrive/legitimate.csv')


In [ ]:
phishing_urls.head()

,Unnamed: 0,URL,label
0,3086811,brielletlrapq.blogspot.mx,1
1,3086812,imgshost.pro,1
2,3086813,ipw.metadsp.co.uk,1
3,3086814,amvoyeur.blogspot.lt,1
4,3086815,edgecommunications.com,1


In [ ]:
legitimate_urls.head()

,Unnamed: 0,URL,label
0,0,karens-krazy-korner.blogspot.com/2010/,0
1,1,www.tmentertainment.com/ccount/click.php?id=173,0
2,2,asian.videos.xblog.in/thai-wife-get-fuck-by-hu...,0
3,3,kpn.se/resurser/bilder-att-farglagga,0
4,4,www.indokitay.ru/landing/toniki-aasha-herbals/,0


In [ ]:
combined_dataset = dd.concat([phishing_urls, legitimate_urls])

In [ ]:
import pandas as pd
import json

# Convert the Dask DataFrame to a Pandas DataFrame
combined_df = combined_dataset.compute()

# Save the combined DataFrame as a JSON file
combined_df.to_json('combined_dataset.json', orient='records', lines=True)


In [ ]:
import json

# Load the combined data from the JSON file
with open('/content/combined_data.json', 'r') as file:
    data = [json.loads(line) for line in file]

In [ ]:
import dask.dataframe as dd

# Load the JSON data into a Dask DataFrame
# Assuming the JSON file is newline-delimited
combined_data = dd.read_json('/content/combined_data.json', lines=True)


**Feature Extraction**

In [ ]:
import dask.dataframe as dd
import pandas as pd
from urllib.parse import urlparse
import re
import socket
from dask.diagnostics import ProgressBar

# Define your feature extraction function
def extract_features(url):
    # Check if the URL is a string
    if not isinstance(url, str):
        # Skip non-string values
        return pd.Series({
            'domain': None,
            'ip_address': None,
            '@_symbol': None,
            'length': None,
            'depth': None,
            'redirection': None,
            'http_https': None,
            'url_shortening': None,
            'prefix_suffix': None
        })

    parsed_url = urlparse(url)
    features = {
        'domain': parsed_url.netloc,
        'ip_address': 0, # Placeholder, will be updated
        '@_symbol': '@' in url,
        'length': len(url),
        'depth': url.count('/'),
        'redirection': '//' in url,
        'http_https': parsed_url.scheme,
        'url_shortening': 'tinyurl' in url.lower(),
        'prefix_suffix': '-' in parsed_url.netloc
    }

    # Check for IP address in the domain
    try:
        socket.gethostbyname(parsed_url.netloc)
        features['ip_address'] = 1
    except socket.gaierror:
        pass

    return pd.Series(features)


# Assuming combined_data is a Dask DataFrame loaded from your JSON file

# Apply the feature extraction function to each URL
with ProgressBar():
    combined_data['features'] = combined_data['URL'].map_partitions(lambda df: df.apply(extract_features), meta=('URL', 'object')).compute()

[####################                    ] | 50% Completed | 38m 26s

In [ ]:
import json

# Load the combined data from the JSON file
with open('/content/combined_data.json', 'r') as file:
    data = [json.loads(line) for line in file]

In [ ]:

# Print the first 5 records
for record in data[:10]:
    print(record)


{'Unnamed: 0': 3086811, 'URL': 'brielletlrapq.blogspot.mx', 'label': 1}
{'Unnamed: 0': 3086812, 'URL': 'imgshost.pro', 'label': 1}
{'Unnamed: 0': 3086813, 'URL': 'ipw.metadsp.co.uk', 'label': 1}
{'Unnamed: 0': 3086814, 'URL': 'amvoyeur.blogspot.lt', 'label': 1}
{'Unnamed: 0': 3086815, 'URL': 'edgecommunications.com', 'label': 1}
{'Unnamed: 0': 3086816, 'URL': 'look-at-all-this-porn.tumblr.com', 'label': 1}
{'Unnamed: 0': 3086817, 'URL': 'mydd.com/users/greg-bloom/posts/strip-mining-the-grassroots-pt-5-grassroots-campaigns-incs-canvassing-model', 'label': 1}
{'Unnamed: 0': 3086818, 'URL': 'adults-entertainment.tumblr.com', 'label': 1}
{'Unnamed: 0': 3086819, 'URL': 'quakes.acmeaom.com', 'label': 1}
{'Unnamed: 0': 3086820, 'URL': 'sexypantyhoseoftheday.blogspot.com.ng', 'label': 1}


In [ ]:
pip install dnspython


In [ ]:
pip install whois

In [ ]:
import dask.dataframe as dd

# Load the JSON data into a Dask DataFrame
# Assuming the JSON file is newline-delimited
combined_data = dd.read_json('/content/combined_dataset.json', lines=True)


In [ ]:
import dns.resolver
import whois
from datetime import datetime
from urllib.parse import urlparse
import pandas as pd
from dask.diagnostics import ProgressBar

def extract_features(url):
    # Parse the URL to get its components
    parsed_url = urlparse(url)

    # Initialize the features dictionary
    features = {}

    # Initialize whois_record to None
    features['whois_record'] = None

    # DNS Record
    try:
        dns_records = dns.resolver.resolve(parsed_url.netloc, 'A')
        features['dns_record'] = [str(record) for record in dns_records]
    except dns.resolver.NoAnswer:
        features['dns_record'] = None

    # Whois Record
    try:
        whois_info = whois.whois(parsed_url.netloc)
        features['whois_record'] = whois_info.creation_date
    except Exception as e:
        pass  # Do nothing if whois lookup fails

    # Age of Domain
    if features['whois_record']:
        creation_date = datetime.strptime(features['whois_record'], '%Y-%m-%d')
        today = datetime.now()
        age = today - creation_date
        features['age_of_domain'] = age.days / 365
    else:
        features['age_of_domain'] = None

    # End Period of Domain
    if features['whois_record']:
        expiration_date = datetime.strptime(features['whois_record'], '%Y-%m-%d')
        today = datetime.now()
        end_period = expiration_date - today
        features['end_period_of_domain'] = end_period.days
    else:
        features['end_period_of_domain'] = None

    return pd.Series(features)

# Assuming combined_data is your Dask DataFrame
# Define the output metadata for the new 'features' column
# This should match the structure of the output from your extract_features function
features_meta = pd.Series({
    'dns_record': 'object',
    'whois_record': 'object',
    'age_of_domain': 'float64',
    'end_period_of_domain': 'float64'
})

# Apply the feature extraction function to each URL
with ProgressBar():
    combined_data['features'] = combined_data['URL'].map_partitions(
        lambda df: df.apply(extract_features),
        meta=features_meta  # Using the features_meta variable defined earlier
    ).compute()

In [ ]:
import json

# Load the combined data from the JSON file
with open('/content/combined_data.json', 'r') as file:
    data = [json.loads(line) for line in file]
# Print the first 5 records
for record in data[:10]:
    print(record)


In [ ]:
import re
from bs4 import BeautifulSoup

# Function to extract HTML and JavaScript-based features
def extract_html_features(html_content):
    features = {}

    # IFrame Redirection
    if re.search(r'<iframe\s', html_content):
        features['iframe_redirection'] = True
    else:
        features['iframe_redirection'] = False

    # Status Bar Customization
    if re.search(r'status\s*=\s*[\'"]?\s*hide\s*[\'"]?', html_content, re.IGNORECASE):
        features['status_bar_customization'] = True
    else:
        features['status_bar_customization'] = False

    # Disabling Right Click
    if re.search(r'oncontextmenu\s*=\s*[\'"]\s*return\s+false\s*[\'"]', html_content, re.IGNORECASE):
        features['disabling_right_click'] = True
    else:
        features['disabling_right_click'] = False

    # Website Forwarding
    if re.search(r'meta\s+http-equiv\s*=\s*[\'"]\s*refresh\s*[\'"]', html_content, re.IGNORECASE):
        features['website_forwarding'] = True
    else:
        features['website_forwarding'] = False

    return features

# Function to extract HTML features from a URL
def extract_html_features_from_url(url):
    # Write your code to fetch HTML content from the URL (using requests, urllib, etc.)
    # For demonstration purposes, let's assume you have the HTML content stored in a variable called 'html_content'
    html_content = "<html>...</html>"  # Placeholder HTML content

    # Extract HTML features
    return extract_html_features(html_content)

# Apply the function to each URL in combined_data
combined_data['html_features'] = combined_data['URL'].map_partitions(
    lambda urls: urls.apply(extract_html_features_from_url),
    meta=('html_features', 'object')
).compute()
